## Project 2: Libraries and Social Media

This project uses the MongoDB library database as a base to create an additional social media capability.

### Database Design

The namespace is called 'library' and contains two collections - one of users, one of the books in the library.

The users collection includes:

- name
- phone
- address
- uni affiliation.

The books collection includes:
- book title
- primary and secondary author
- publisher date
- number of pages
- the publisher
- translator 
- topic 

For books that have been checked out, include a checked flag along with additional information on: checked date, user who checked the book, including their university affiliation.

For books with comments, there are also information containing the comments the borrower has posted about the book.

In [102]:
import json
from pymongo import MongoClient

In [104]:
client = MongoClient()
print(client)
db = client.library

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


## Query 1

Which books have been checked out since such and such date?

In [119]:
books = db.books.find(
    {'checked': "Yes", 'checkdate': { '$gte': "2018-01-01" }, 'checkdate': { '$lte': "2020-02-01" }}, 
    {'borrower': 1, 'name': 1, 'checkdate': 1})

for book in books:
    print(book)

{'_id': ObjectId('5ea39e3ffebbdbf5a7150b84'), 'name': 'Introduction to Machine Learning', 'checkdate': '2020-01-01', 'borrower': 'Adam Apple'}
{'_id': ObjectId('5ea39f06febbdbf5a7150b85'), 'name': 'Intermediate Machine Learning', 'checkdate': '2020-01-01', 'borrower': 'Adam Apple'}


## Query 2

Which users have check out such and such book?

In [121]:
books = db.books.find(
    {'checked': "Yes"},
    {'borrower': 1, 'name': 1}
)
for book in books:
    print(book)

{'_id': ObjectId('5ea39e3ffebbdbf5a7150b84'), 'name': 'Introduction to Machine Learning', 'borrower': 'Adam Apple'}
{'_id': ObjectId('5ea39f06febbdbf5a7150b85'), 'name': 'Intermediate Machine Learning', 'borrower': 'Adam Apple'}
{'_id': ObjectId('5ea39f40febbdbf5a7150b87'), 'name': 'Midsummer Murders', 'borrower': 'Dennis Donuts'}


## Query 3

How many books does the library have on such and such topic?

In [124]:
books = db.books.aggregate([
    {
        '$bucket': {
            'groupBy': "$topic",
            'boundaries': ["Fiction", "Machine Learning", "Psychology", "Upper bound placeholder"],
            'default': "All others",
            'output': {'count': {'$sum': 1}}
        }
    }
])

for book in books:
    print(book)

{'_id': 'Fiction', 'count': 1}
{'_id': 'Machine Learning', 'count': 3}
{'_id': 'Psychology', 'count': 1}


## Query 4

Which users from Columbia University have checked out books on Machine Learning between this date and that date?

In [125]:
books = db.books.find(
    {'checked': "Yes", 
    'borrower_uni': "Columbia University", 
    'checkdate': { '$gte': "2018-01-01" }, 
    'checkdate': { '$lte': "2020-02-01" }},
    {'borrower': 1, 'checkdate': 1}
)

for book in books:
    print(book)

{'_id': ObjectId('5ea39e3ffebbdbf5a7150b84'), 'checkdate': '2020-01-01', 'borrower': 'Adam Apple'}
{'_id': ObjectId('5ea39f06febbdbf5a7150b85'), 'checkdate': '2020-01-01', 'borrower': 'Adam Apple'}


## Query 5

What comments have been made by any User about such and such book between such and such dates, ordered from the most recent to the least recent?

In [134]:
comments = db.books.find(
    {'checked': "Yes", 
    'checkdate': { '$gte': "2018-01-01" }, 
    'checkdate': { '$lte': "2020-04-01" }},
    {'title': 1, 'borrower': 1, 'comments': 1, 'checkdate': 1}
).sort('checkdate')

for comment in comments:
    print(comment)

{'_id': ObjectId('5ea39e3ffebbdbf5a7150b84'), 'checkdate': '2020-01-01', 'borrower': 'Adam Apple', 'comments': ['This is a great book', 'I learnt a lot', 'I recommend!']}
{'_id': ObjectId('5ea39f06febbdbf5a7150b85'), 'checkdate': '2020-01-01', 'borrower': 'Adam Apple', 'comments': ['This is also a great book', 'I also learnt a lot', 'I defs recommend!']}
{'_id': ObjectId('5ea39f40febbdbf5a7150b87'), 'checkdate': '2020-03-01', 'borrower': 'Dennis Donuts', 'comments': ['This is an interesting read', 'Great characters', 'I rate 5/5']}


## Query 6

Show for a given User, what comments they have made about such and such book.

In [136]:
comments = db.books.find({'checked': "Yes"}, {'borrower': 1, 'comments': 1})

for comment in comments:
    print(comment)

{'_id': ObjectId('5ea39e3ffebbdbf5a7150b84'), 'borrower': 'Adam Apple', 'comments': ['This is a great book', 'I learnt a lot', 'I recommend!']}
{'_id': ObjectId('5ea39f06febbdbf5a7150b85'), 'borrower': 'Adam Apple', 'comments': ['This is also a great book', 'I also learnt a lot', 'I defs recommend!']}
{'_id': ObjectId('5ea39f40febbdbf5a7150b87'), 'borrower': 'Dennis Donuts', 'comments': ['This is an interesting read', 'Great characters', 'I rate 5/5']}
